In [2]:
import numpy as np
import pandas as pd
import os

In [11]:
df_training = pd.read_csv('train_features.csv')
df_training_label = pd.read_csv('train_labels.csv')
all_pids = [pid for pid in df_training['pid'].unique()]


In [120]:
def getPatientData(trainingData, pids):
    patientTrainingDataIndex = [trainingData.iloc[idx] for idx, col in enumerate(trainingData) if trainingData['pid'][idx] in pids]
    #patientTrainingDataIndex = trainingData[trainingData['pid'] == pids]
    
    return pd.DataFrame(patientTrainingDataIndex)

def partitionData(trainingDataPids, trainingPartition=80):
    validationPartition = 100 - trainingPartition
    countTraining = int((trainingPartition/100)*len(trainingDataPids))
    training = trainingDataPids[:countTraining]
    validation = trainingDataPids[countTraining:]
    print('Training size: ' + str(countTraining))
    print('Validation size: ' + str(len(validation)))
    return training, validation


In [124]:
res = getPatientData(df_training[:100], [1])
trainingPIDS, validationPIDS = partitionData(all_pids)


Training size: 15196
Validation size: 3799


In [127]:
for singlePID in trainingPIDS:
    print(singlePID)
    X_pid = getPatientData(df_training, [singlePID])
    print(X_pid)
    break

1
    pid  Time   Age  EtCO2  PTT   BUN  Lactate  Temp  Hgb  HCO3  ...  \
0   1.0   3.0  34.0    NaN  NaN  12.0      NaN  36.0  8.7  24.0  ...   
1   1.0   4.0  34.0    NaN  NaN   NaN      NaN  36.0  NaN   NaN  ...   
2   1.0   5.0  34.0    NaN  NaN   NaN      NaN  36.0  NaN   NaN  ...   
3   1.0   6.0  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
4   1.0   7.0  34.0    NaN  NaN   NaN      NaN   NaN  NaN   NaN  ...   
5   1.0   8.0  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
6   1.0   9.0  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
7   1.0  10.0  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
8   1.0  11.0  34.0    NaN  NaN  12.0      NaN   NaN  8.5  26.0  ...   
9   1.0  12.0  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
10  1.0  13.0  34.0    NaN  NaN   NaN      NaN   NaN  NaN   NaN  ...   
11  1.0  14.0  34.0    NaN  NaN   NaN      NaN   NaN  NaN   NaN  ...   

    Alkalinephos   SpO2  Bilirubin_direct  Chloride   Hct  He